In [2]:
import numpy as np
import sys

In [4]:
a = sys.maxsize

In [15]:
# a 64位，二进制63个1

In [22]:
#-a >> 65 仍然是-1

-1

## 1. 最长回文子序列长度
* 可以不连续，正反一样
* 例如：a12b3c43def2ghi1kpm，最长为1234321或123c123，返回7
* 链接：https://leetcode.com/problems/longest-palindromic-subsequence/
* 思路1：一个字符串和他的逆序的最长公共子序列，就是最长回文子序列
* 思路2：以下

In [25]:
#递归，从L到R的最长回文子序列长度
def process(stringList, L, R):
    #只有一个字符，本身就是回文
    if L == R:
        return 1
    if L == R-1:
        return 2 if stringList[L] == stringList[R] else 1
    #情况1，不以L开头，也不以R结尾
    #情况2，以L开头，不以R结尾
    #情况3，不以L开头，以R结尾
    #情况4，以L开头，也以R结尾
    p1 = process(stringList, L+1, R-1)
    p2 = process(stringList, L, R-1)
    p3 = process(stringList, L+1, R)
    p4 = process(stringList, L+1, R-1) + 2 if stringList[L] == stringList[R] else 0
    return max([p1,p2,p3,p4])
    
def longestPalindromeSubseq(string):
    if not string:
        return 0
    ans = process(list(string), 0, len(string)-1)
    return ans

In [32]:
def longestPalindromeSubseq2(s):
    if not s:
        return 0
    stringList = list(s)
    N = len(s)
    dp = [[0] * N for _ in range(N)]
    for i in range(N):
        dp[i][i] = 1
    for i in range(1, N):
        dp[i-1][i] = 2 if stringList[i-1] == stringList[i] else 1
    for L in range(N-3, -1, -1):
        for R in range(L+2, N):
            p1 = dp[L+1][R-1]
            if stringList[L] == stringList[R]:
                p1 += 2 #p4改
            p2 = dp[L][R-1]
            p3 = dp[L+1][R]
            dp[L][R] = max([p1,p2,p3])
    return dp[0][N-1]

In [34]:
# 继续优化
# [i]依赖三个变量
# 左 <- [i]
# 左下↙ ↓     ==> i不可能比三个方向的小，所以左不可能比左下的小、下也不可能比左下的小。所以左下可以省略
#       下
def longestPalindromeSubseq3(s):
    if not s:
        return 0
    stringList = list(s)
    N = len(s)
    dp = [[0] * N for _ in range(N)]
    for i in range(N):
        dp[i][i] = 1
    for i in range(1, N):
        dp[i-1][i] = 2 if stringList[i-1] == stringList[i] else 1
    for L in range(N-3, -1, -1):
        for R in range(L+2, N):
            #优化
            dp[L][R] = max(dp[L][R-1], dp[L+1,R])
            if stringList[L] == stringList[R]:
                dp[L][R] = max(dp[L][R], 2+dp[L+1][R-1])
    return dp[0][N-1]

## 2. 棋盘
* 马走日，从(0,0)出发，步数=k，返回方法数
* 棋盘大小10*9

In [36]:
# 当前来到x,y
# 还剩rest步，返回正好跳到a,b的方法数是多少？
def process(x, y, rest, a, b):
    if x < 0 or x > 9 or y < 0 or y > 8:
        return 0
    if rest == 0:
        return 1 if x==a and y==b else 0
    #8种蹦法
    ways =  process(x+2,y+1,rest-1,a,b)
    ways += process(x+1,y+2,rest-1,a,b)    
    ways += process(x-1,y+2,rest-1,a,b)    
    ways += process(x-2,y+1,rest-1,a,b)    
    ways += process(x-2,y-1,rest-1,a,b)    
    ways += process(x-1,y-2,rest-1,a,b)    
    ways += process(x+1,y-2,rest-1,a,b)    
    ways += process(x+2,y-1,rest-1,a,b) 
    return ways

def jump(a, b, k):
    return process(0,0,k,a,b)

In [37]:
#防越界小技巧
def pick(dp, x, y, rest):
    if x < 0 or x > 9 or y < 0 or y > 8:
        return 0
    else:
        return dp[x][y][rest]

def jump2(a,b,k):
    dp = [[[0] * (k+1) for _ in range(9)] for _ in range(10)]
    dp[a][b][0] = 1
    #8种蹦法，只依赖下一层！
    for rest in range(1, k+1):
        for x in range(10):
            for y in range(9):
                ways = pick(dp, x+2,y+1,rest-1)
                ways += pick(dp,x+1,y+2,rest-1)    
                ways += pick(dp,x-1,y+2,rest-1)    
                ways += pick(dp,x-2,y+1,rest-1)    
                ways += pick(dp,x-2,y-1,rest-1)    
                ways += pick(dp,x-1,y-2,rest-1)    
                ways += pick(dp,x+1,y-2,rest-1)    
                ways += pick(dp,x+2,y-1,rest-1)
                dp[x][y][res] = ways
    return dp[0][0][k]

## 3. 煮咖啡
* 给定一个数组，arr[i]代表第i号咖啡机泡一杯咖啡的时间
* 给定一个正数N，表示N个人在等咖啡，每台咖啡机只能同时泡一杯咖啡，咖啡机可以并行泡
* 只有一台洗杯机，一次只能洗一个杯子，耗时a，洗完才能洗下一杯
* 每个杯子也可以自己挥发干净，耗时b，咖啡杯可以并行挥发
* 假设所有人拿到咖啡立刻喝干净，不耗时。返回从开始到所有咖啡机变干净的最短时间。
* 四个参数arr，N，a，b

In [63]:
'''
简化问题：所有人拿到咖啡的时间尽量小
思路： 小根堆作为选择咖啡机的标准，存储的对象为(咖啡机拿来使用的时间节点，泡咖啡需要多久)，排序标准为使用时间节点+泡多久
      实际是用小根堆弹压操作，模拟所有人一块排队的过程
'''
import heapq
class Machine:
    def __init__(self, timePoint = None, workTime=None):
        self.timePoint = timePoint
        self.workTime = workTime
    def __lt__(self, other):
        return (self.timePoint+self.workTime) < (other.timePoint+other.workTime)
    
    def __eq__(self, other):
        return (self.timePoint+self.workTime) == (other.timePoint+other.workTime)
    
    def __le__(self, other):
        return (self.timePoint+self.workTime) <= (other.timePoint+other.workTime)
    
    def __str__(self):
        return "timePoint: {}, workTime: {}".format(self.timePoint, self.workTime)
    
def minTime(arr, n, a, b):
    heap = list()
    for i in range(len(arr)):
        heapq.heappush(heap, Machine(0, arr[i]))
    drinks = [0] * n
    for i in range(n):
        cur = heapq.heappop(heap)
        cur.timePoint += cur.workTime
        drinks[i] = cur.timePoint
        heapq.heappush(heap,cur)
#     print(drinks) 喝完的最优数组
    #怎么洗
#     ans = bestWashTime(drinks, wash, air, 0, 0) 递归调用。所以dp返回[0][0]
    ans = bestWashTimeDp(drinks, wash, air)
    return ans
    
# 暴力洗
# drinks待洗的杯子时间点；wash洗杯子所需时间，air晾干所需时间，洗到第index个杯子，洗机下次可用时间
# 返回drinks内所有杯子都变干净的时间
def bestWashTime(drinks, wash, air, index, free):
    if index == len(drinks):
        return 0
    # index号杯子 决定洗
    #index 依赖 index + 1， 改dp时从下往上填
    selfClean1 = max(drinks[index], free) + wash #max(喝完的时间、洗机空闲时间)=开始洗的时间 + 洗的时间
    restClean1 = bestWashTime(drinks, wash, air, index+1, selfClean1) #selfClean1传入，前面洗的影响后面的时间
    p1 = max(selfClean1, restClean1)
    #index号杯子 决定挥发
    selfClean2 = drinks[index] + air
    restClean2 = bestWashTime(drinks, wash, air, index+1, free)
    p2 = max(selfClean2, restClean2)
    return min(p1,p2)


#当某一个变量不知道范围，人为靠业务知识限制范围。这里free的最大值，是所有杯子都去洗
def bestWashTimeDp(drinks, wash, air):
    N = len(drinks)
    maxFree = 0
    for i in drinks:
        maxFree = max(i, maxFree) + wash
    dp = [[0] * (maxFree+1) for _ in range(N+1)]
    #dp[N][...] = 0 对应 index == len(drinks): return 0
    for index in range(N-1, -1, -1):
        for free in range(maxFree+1):
            # index号杯子 决定洗
            #index 依赖 index + 1， 改dp时从下往上填
            selfClean1 = max(drinks[index], free) + wash #max(喝完的时间、洗机空闲时间)=开始洗的时间 + 洗的时间
            if selfClean1 > maxFree: #防止dp越界
                continue 
            restClean1 = dp[index+1][selfClean1]
            p1 = max(selfClean1, restClean1)
            
            #index号杯子 决定挥发
            selfClean2 = drinks[index] + air
            restClean2 = dp[index+1][free]
            p2 = max(selfClean2, restClean2)
            dp[index][free] = min(p1,p2)
            
    return dp[0][0] #0位置出发，最早的时间是0

In [64]:
minTime([3,1,7],10,0,0)

[1, 2, 3, 3, 4, 5, 6, 6, 7, 7]
